In [3]:
import psycopg2
import pandas as pd
from sklearn.datasets import fetch_california_housing

In [4]:
# DB connection

db_connect = psycopg2.connect(
    user = "myuser",
    password = "mypassword",
    host = "localhost",
    port = 5432,
    database = "mydatabase"
)

In [5]:
# load dataset (carlifornia housing)

X,y = fetch_california_housing(return_X_y = True, as_frame = True)
df = pd.concat([X,y], axis = 'columns')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [9]:
# table creation query

creation_query = """create table if not exists carlifornia_housing (
    id serial primary key,
    timestamp timestamp,
    MedInc float8,
    HouseAge float8,
    AveRooms float8,
    AveBedrms float8,
    Population float8,
    AveOccup float8,
    Latitude float8,
    Longitude float8,
    MedHouseVal float8
    );"""

In [10]:
# send query

with db_connect.cursor() as cur:
    cur.execute(creation_query)
    db_connect.commit()

# Data Insertion

In [13]:
data = df.sample(1).squeeze()
print(data)

MedInc           3.333300
HouseAge        45.000000
AveRooms         4.594132
AveBedrms        0.965770
Population     946.000000
AveOccup         2.312958
Latitude        34.050000
Longitude     -118.360000
MedHouseVal      2.547000
Name: 4732, dtype: float64


In [14]:
insert_row_query = f"""
insert into carlifornia_housing (
    timestamp,
    MedInc,
    HouseAge,
    AveRooms,
    AveBedrms,
    Population,
    AveOccup,
    Latitude,
    Longitude,
    MedHouseVal
)
values (
    now(),
    {data.MedInc},
    {data.HouseAge},
    {data.AveRooms},
    {data.AveBedrms},
    {data.Population},
    {data.AveOccup},
    {data.Latitude},
    {data.Longitude},
    {data.MedHouseVal}
);
"""

In [15]:
with db_connect.cursor() as cur:
    cur.execute(insert_row_query)
    db_connect.commit()

In [16]:
del db_connect